## Neural Network Model

Use 1D-KNN

Note: This code have to be run after preprocessing.ipynb

In [30]:
import numpy as np
import pandas as pd
import os
import pickle
from pathlib import Path
from keras import backend as K
import matplotlib.pyplot as plt

In [6]:
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras import regularizers
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight
from keras.layers import LeakyReLU

In [ ]:
trainset = pickle.load(open("./trainset.p",'rb'))
testset = pickle.load(open("./testset.p",'rb'))

In [58]:
trainset.groupby('stripPVC').count()

,R_interval_mean,R_interval_sd,age,annotation,gender,id,lead,p2p_mean,p2p_std,peak,strip,p2p_diff_form_norm
stripPVC,,,,,,,,,,,,
False,4046,4046,4046,4046,4046,4046,4046,4046,4046,4046,4046,4046
True,2434,2434,2434,2434,2434,2434,2434,2434,2434,2434,2434,2434


In [57]:
USE_KFOLD = True
testset.groupby('stripPVC').count()

,R_interval_mean,R_interval_sd,age,annotation,gender,id,lead,p2p_mean,p2p_std,peak,strip,p2p_diff_form_norm
stripPVC,,,,,,,,,,,,
False,1379,1379,1379,1379,1379,1379,1379,1379,1379,1379,1379,1379
True,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141


In [59]:
xtrain = trainset['strip']
ytrain = [1 if i == True else 0 for i in trainset['stripPVC']]
xtest = testset['strip']
ytest = [1 if i == True else 0  for i in testset['stripPVC']]

In [60]:
# convert

xtrain = np.stack(xtrain)
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))
xtest = np.stack(xtest)
xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
xtrain.shape




(6480, 1800, 1)

### 1D-CNN

In [61]:
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())
def CNN_2(input_shape, blocks = 2, dropout = 0, regularization = 0.001, kernel_size = 16):  
    classifier = Sequential()
    classifier.add(Conv1D(filters = 100, kernel_size = kernel_size, activation = 'relu', input_shape = (input_shape, 1)))
    classifier.add(Dropout(dropout))
    classifier.add(Conv1D(filters = 160, kernel_size = kernel_size, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 2))
    classifier.add(LeakyReLU())
    classifier.add(Dropout(dropout))
    classifier.add(LeakyReLU())
    classifier.add(GlobalAveragePooling1D())
    classifier.add((Dense(1, activation='sigmoid')))
   
    
 
    
    classifier.compile(optimizer = 'adam',
                       loss = 'binary_crossentropy',
                       metrics = [sensitivity,specificity,'accuracy'])
    return classifier

In [62]:
MODEL_NUMBER = '1'
MODEL = CNN_2
BATCH_SIZE = 10
EPOCHS = 10 


In [ ]:

# TODO Cross Validation

In [63]:
params = { 'blocks': 5, 'dropout': 0.1, 'regularization': 0, 'kernel_size': 16, 'reduceLR': True}
classifier = MODEL(input_shape=np.shape(xtrain)[1], blocks = params['blocks'], dropout=params['dropout'], regularization=params['regularization'])
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)]

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(ytrain),
                                                  ytrain)
history = classifier.fit(xtrain,
                         ytrain,
                         batch_size = BATCH_SIZE,
                         epochs = EPOCHS,
                         validation_split=0.2,
                         class_weight = class_weights,
                         callbacks = callbacks
                        )

Train on 5184 samples, validate on 1296 samples
Epoch 1/10
5184/5184 [==============================] - 246s 47ms/step - loss: 0.5652 - sensitivity: 0.3602 - specificity: 0.8996 - acc: 0.7052 - val_loss: 0.7554 - val_sensitivity: 0.5208 - val_specificity: 0.5008 - val_acc: 0.6829
Epoch 2/10
5184/5184 [==============================] - 251s 48ms/step - loss: 0.4892 - sensitivity: 0.5367 - specificity: 0.9112 - acc: 0.7805 - val_loss: 0.9304 - val_sensitivity: 0.5763 - val_specificity: 0.4844 - val_acc: 0.7114
Epoch 3/10
5184/5184 [==============================] - 267s 52ms/step - loss: 0.4641 - sensitivity: 0.5750 - specificity: 0.9250 - acc: 0.7996 - val_loss: 0.6319 - val_sensitivity: 0.4502 - val_specificity: 0.6068 - val_acc: 0.7130
Epoch 4/10
5184/5184 [==============================] - 243s 47ms/step - loss: 0.4497 - sensitivity: 0.6129 - specificity: 0.9327 - acc: 0.8166 - val_loss: 0.6320 - val_sensitivity: 0.4128 - val_specificity: 0.6140 - val_acc: 0.6898
Epoch 5/10
5184/5184

In [64]:
trainpred = classifier.predict_classes(xtrain)
testpred = classifier.predict_classes(xtest)

In [65]:
def metric(pred, real):
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import roc_auc_score
    ConfusionMatrix= pd.DataFrame(confusion_matrix(real,pred), columns=['P(y=1)','P(y=0)'], index=['True','False'])
    print(ConfusionMatrix)
    print('----------------')
    print('Precision:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0]))
    print('Recall:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]))
    print('Accuracy:', (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]) / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0] + ConfusionMatrix.iloc[0,1] + ConfusionMatrix.iloc[1,1]))
    print('AUC:', roc_auc_score(pred,real))

In [66]:
metric([t[0] for t in testpred], ytest)


       P(y=1)  P(y=0)
True     1242     137
False     700     441
----------------
Precision: 0.639546858908342
Recall: 0.7379679144385026
Accuracy: 0.6678571428571428
AUC: 0.7012613187275274


### Integrate with randomforest

In [68]:
from sklearn import preprocessing

col = trainset.drop(['peak','annotation','strip','stripPVC','id','lead'],axis=1).columns.values
scale = preprocessing.StandardScaler()
scale.fit(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1))
trainset['cnnProb'] =[t[0] for t in trainpred]
testset['cnnProb'] = [t[0] for t in testpred]
n_trainset = pd.DataFrame(scale.transform(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)
n_testset = pd.DataFrame(scale.transform(testset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)


In [74]:
from sklearn.ensemble import RandomForestClassifier
for i in range(5,8):
    clf = RandomForestClassifier(n_estimators=100, max_depth=3,max_features=i, random_state=0)
    clf.fit(n_trainset , trainset['stripPVC'])
    pred = clf.predict(n_testset)
    print('max features: ' , i)
    print(metric(pred,testset['stripPVC']))

max features:  5
       P(y=1)  P(y=0)
True     1320      59
False     430     711
----------------
Precision: 0.7542857142857143
Recall: 0.6499261447562777
Accuracy: 0.805952380952381
AUC: 0.8388311688311688
None
max features:  6
       P(y=1)  P(y=0)
True     1291      88
False     429     712
----------------
Precision: 0.7505813953488372
Recall: 0.6445332001997004
Accuracy: 0.7948412698412698
AUC: 0.8202906976744186
None
max features:  7
       P(y=1)  P(y=0)
True     1287      92
False     430     711
----------------
Precision: 0.749563191613279
Recall: 0.6441441441441441
Accuracy: 0.7928571428571428
AUC: 0.8174964152337877
None
